In [160]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [161]:
data= pd.read_csv('~/code/dmassonfr/Spotify_hit_predictor/raw_data/tracks.csv')

In [162]:
df= pd.read_csv('~/code/dmassonfr/Spotify_hit_predictor/raw_data/artists.csv')

In [163]:
data.shape

(586672, 20)

In [164]:
df.shape

(1162095, 5)

In [165]:
data.describe()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,586672.000000,5.866720e+05,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000
mean,27.570053,2.300512e+05,0.044086,0.563594,0.542036,5.221603,-10.206067,0.658797,0.104864,0.449863,0.113451,0.213935,0.552292,118.464857,3.873382
std,18.370642,1.265261e+05,0.205286,0.166103,0.251923,3.519423,5.089328,0.474114,0.179893,0.348837,0.266868,0.184326,0.257671,29.764108,0.473162
min,0.000000,3.344000e+03,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.000000,1.750930e+05,0.000000,0.453000,0.343000,2.000000,-12.891000,0.000000,0.034000,0.096900,0.000000,0.098300,0.346000,95.600000,4.000000
50%,27.000000,2.148930e+05,0.000000,0.577000,0.549000,5.000000,-9.243000,1.000000,0.044300,0.422000,0.000024,0.139000,0.564000,117.384000,4.000000
75%,41.000000,2.638670e+05,0.000000,0.686000,0.748000,8.000000,-6.482000,1.000000,0.076300,0.785000,0.009550,0.278000,0.769000,136.321000,4.000000
max,100.000000,5.621218e+06,1.000000,0.991000,1.000000,11.000000,5.376000,1.000000,0.971000,0.996000,1.000000,1.000000,1.000000,246.381000,5.000000


In [166]:
#Creating release_year column to sample data by year of release

data['release_year'] = [int(i.split('-')[0]) for i in data['release_date']]

In [167]:
#Sampling the data

data = data[data['release_year']>=2000]

In [168]:
#Dropping non musical data

data = data[data['speechiness']<0.66]

In [169]:
data.head(1)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_year
39501,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,['Gerry & The Pacemakers'],['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,-11.101,1,0.0322,0.394,0.0,0.149,0.285,113.564,3,2008


In [170]:
df.head(1)

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0


In [171]:
# merge tracks and artist dataframes
df.rename(columns = {'id': 'id_artists','popularity': 'artists_popularity'}, inplace = True)
df.drop(['name'],axis = 1, inplace = True)

data['id_artists'] = [i[2:-2] for i in data['id_artists']]

In [172]:
df.shape

(1162095, 4)

In [173]:
data= data.merge(df, on='id_artists')

In [174]:
#Drop non numerical columns

X= data.drop(columns=['id', 'name', 'artists','id_artists','release_date'])

In [175]:
y= data['popularity']

In [176]:
#Visualize features correlation

corr = X.corr()
corr.style.background_gradient(cmap='coolwarm')

/tmp/ipykernel_31238/2794285327.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = X.corr()


,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_year,followers,artists_popularity
popularity,1.000000,-0.052331,0.141058,0.096633,-0.037625,-0.001226,0.107148,0.002760,0.044624,0.017051,-0.221823,-0.076977,-0.007709,-0.013960,0.010354,0.144299,0.188992,0.580072
duration_ms,-0.052331,1.000000,-0.056915,-0.112019,-0.007964,0.006163,-0.018147,-0.008263,-0.076361,-0.023826,0.066574,0.028316,-0.152579,-0.003575,0.001988,-0.084774,-0.012773,-0.049137
explicit,0.141058,-0.056915,1.000000,0.154023,0.049699,0.006078,0.050954,-0.056058,0.377740,-0.079537,-0.059923,-0.012018,-0.016767,-0.022803,0.034510,0.169694,0.074529,0.141862
danceability,0.096633,-0.112019,0.154023,1.000000,0.116475,0.019191,0.153542,-0.070002,0.184727,-0.123214,-0.131429,-0.154089,0.487612,-0.140141,0.147589,0.105032,0.034406,0.031252
energy,-0.037625,-0.007964,0.049699,0.116475,1.000000,0.030733,0.723356,-0.082083,0.132627,-0.686523,-0.056347,0.165042,0.382758,0.176624,0.161578,-0.022032,0.019509,0.021303
key,-0.001226,0.006163,0.006078,0.019191,0.030733,1.000000,0.022211,-0.134809,0.027200,-0.020115,-0.003568,-0.003802,0.027655,0.002874,0.009243,0.000292,-0.006600,-0.005225
loudness,0.107148,-0.018147,0.050954,0.153542,0.723356,0.022211,1.000000,-0.037032,0.052337,-0.503688,-0.314907,0.053538,0.280284,0.142847,0.136079,0.020342,0.082237,0.118486
mode,0.002760,-0.008263,-0.056058,-0.070002,-0.082083,-0.134809,-0.037032,1.000000,-0.082479,0.070193,-0.026698,0.009055,-0.027931,0.013501,-0.023800,-0.041572,0.000431,0.007799
speechiness,0.044624,-0.076361,0.377740,0.184727,0.132627,0.027200,0.052337,-0.082479,1.000000,-0.083414,-0.057479,0.059604,0.092817,0.016846,0.034643,0.136393,0.031900,0.078730
acousticness,0.017051,-0.023826,-0.079537,-0.123214,-0.686523,-0.020115,-0.503688,0.070193,-0.083414,1.000000,0.020446,-0.062041,-0.191087,-0.133801,-0.144747,-0.015436,-0.059317,-0.045739


In [177]:
#Droped features with less 5% correlation

X= X.drop(columns=['valence','speechiness','key','duration_ms','mode','liveness','popularity','genres'])

In [178]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

scaler= MinMaxScaler()

#Fit the model into X_train
X_train_scaled = scaler.fit(X_train)

#Transform X_train data
X_train_scaled = scaler.transform(X_train)

# Change X_train_scaled to a Dataframe
X_train_scaled = pd.DataFrame(X_train_scaled)

#Fit the model into X_test
X_test_scaled = scaler.fit(X_test)

#Transform X_test data
X_test_scaled = scaler.transform(X_test)

# Change X_test_scaled to a Dataframe
X_test_scaled = pd.DataFrame(X_test)

In [181]:
#from sklearn.model_selection import GridSearchCV
#from sklearn.neighbors import LogisticRegressor
#from sklearn.metrics import r2_score, mean_absolute_error

# Hyperparameter Grid
#n_neighbors = range(5,101,5)
#p = [1,2,3]

# Instantiate model
#model = KNeighborsRegressor()

# Instantiate Grid Search
#grid = GridSearchCV(estimator = model,
#                    param_grid = {'n_neighbors': n_neighbors, 'p': p},
#                    scoring='r2',
#                    cv=5,
#                    n_jobs=-1)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
model = LogisticRegression(solver='sag')

cv_sgd = cross_validate(
    model,
    X_train_scaled, y_train,
    cv = 10,
    scoring = 'r2')


/home/antonio/.pyenv/versions/3.10.6/envs/Spotify_hit_predictor/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


In [ ]:
# Fit data to Grid Search
#grid.fit(X_train_scaled, y_train)

In [ ]:
#grid.best_params_

In [ ]:
#grid.best_score_